In [ ]:
proj_path = '/home/ajhnam/projects/hidden_singles_public/'
pickle_path = '/data2/pdp/ajhnam/hidden_singles_public/'

In [ ]:
import sys
sys.path.append(proj_path + 'python/')

import random
import numpy as np
import itertools
import pandas as pd
import math
from tqdm.auto import tqdm
from scipy.stats import truncnorm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Dirichlet, Bernoulli, Categorical, Uniform
from torch.utils.data import DataLoader as DataLoader

from hiddensingles.misc import torch_utils as tu
from hiddensingles.misc import utils, TensorDict, TensorDictDataset, nnModule, MLP

In [ ]:
device = 0

In [ ]:
def load_responses(puzzle_data_path, subject_data_path, solvers=True, device='cpu'):
    """
    Returns a tensor of shape [num_solvers, num_trials]
    containing the response types of each subject for each trial
    """
    puzzle_df = pd.read_csv(proj_path + puzzle_data_path, sep='\t')
    subject_df = pd.read_csv(proj_path + subject_data_path, sep='\t')
    subject_df = subject_df[~subject_df.solver.isna()]
    subject_df.solver = subject_df.solver.astype(bool)
    subject_df = subject_df[subject_df.solver if solvers else ~subject_df.solver]
    subject_ids = set(subject_df.subject_id)

    response_map = {'inhouse': 0,
                    'absent': 1,
                    'distractor': 2,
                    'target': 3}
    puzzle_df = puzzle_df[(puzzle_df.phase == 1) & (puzzle_df.subject_id.isin(subject_ids))]
    num_trials = len(puzzle_df.trial.unique())
    puzzle_df.response_type = [response_map[response] for response in puzzle_df.response_type]
    puzzle_df = puzzle_df.sort_values(['subject_id', 'trial'])
    responses = torch.tensor(puzzle_df.response_type.to_numpy(), device=device).view(-1, num_trials)
    subject_ids = puzzle_df.subject_id.unique()
    return responses, subject_ids

In [ ]:
def get_p_strat_responses(p_lapse_ih, p_lapse_a, p_lapse_d):
    """
    Returns P(response | strategy), i.e. probabilities of each response type given strategy
    """
    device = p_lapse_ih.device
    p = torch.tensor([[3/9, 4/9, 1/9, 1/9],
                      [1,   4/6, 1/6, 1/6],
                      [1,   1,   1/2, 1/2],
                      [1,   1,   1,   1  ]], device=device)
    
    # for gradient to flow, need to assign values like this
    multipliers = [[1,          1,              1,                          1],
                   [p_lapse_ih, 1 - p_lapse_ih, 1 - p_lapse_ih,             1 - p_lapse_ih],
                   [p_lapse_ih, p_lapse_a,      1 - p_lapse_ih - p_lapse_a, 1 - p_lapse_ih - p_lapse_a],
                   [p_lapse_ih, p_lapse_a,      p_lapse_d,                  1 - p_lapse_ih - p_lapse_a - p_lapse_d]]
    multipliers = tu.make_tensor(multipliers, dtype=torch.float32, device=device)
    p = p * multipliers
    return p
    

def zero_p_transitions(p_transitions):
    assert p_transitions.shape == (4, 4)
    M = torch.zeros_like(p_transitions)
    for i in range(0, 4):
        M[i, i:] = p_transitions[i, i:].softmax(-1)
    return M

def get_p_strategies(subject_responses,
                     p_strat_trial0,
                     p_transitions):
    """
    Returns P(strategy | trial)
    First, gets P(strategy | trial=1) by solving for
        P(response | strategy) * P(strategy) = P(data)
        at trial = 1.
    Then uses transition matrix to get subsequent state distributions under Markov assumption.
    """
    num_subjects, num_trials = subject_responses.shape
    p_s = p_strat_trial0
    p_strategies = [p_s]
    
    # use transition matrix to derive P(strategy | trial)
    for i in range(num_trials-1):
        p_s = p_transitions.T.matmul(p_s)
        p_strategies.append(p_s)
    p_strategies = torch.stack(p_strategies, dim=0)
    return p_strategies

def get_p_responses(p_strat_responses, p_strategies):
    """
    Returns P(response | trial) by computing
        sum{strategy} P(response | strategy, trial) * P(strategy | trial)
    """
    return p_strategies.matmul(p_strat_responses)

In [ ]:
class MacroModel(nn.Module):
    
    def __init__(self,
                 subject_responses,
                 p_lapse_ih=0.1,
                 p_lapse_a=0.1,
                 p_lapse_d=0.1):
        # cap p_lapse at 20%, though it wouldn't actually reach it, just to keep things somewhat reasonable
        assert 0 < p_lapse_ih <= .2
        assert 0 < p_lapse_a <= .2
        assert 0 < p_lapse_d <= .2
        
        super().__init__()
        self.subject_responses = subject_responses
        
        self.p_lapse_ih = nn.Parameter(tu.logit(torch.tensor(5 * p_lapse_ih))) # max it at 20%
        self.p_lapse_a = nn.Parameter(tu.logit(torch.tensor(5 * p_lapse_a))) # max it at 20%
        self.p_lapse_d = nn.Parameter(tu.logit(torch.tensor(5 * p_lapse_d))) # max it at 20%
        self.p_strat_trial0 = nn.Parameter(torch.rand(4))
        self.p_transitions = nn.Parameter(torch.rand(4, 4))
        
    def get_params(self, detach=True):
        p_lapse_ih = self.p_lapse_ih.sigmoid() / 5
        p_lapse_a = self.p_lapse_a.sigmoid() / 5
        p_lapse_d = self.p_lapse_d.sigmoid() / 5
        p_strat_responses = get_p_strat_responses(p_lapse_ih, p_lapse_a, p_lapse_d)
        
        params = TensorDict(p_lapse_ih = p_lapse_ih,
                            p_lapse_a = p_lapse_a,
                            p_lapse_d = p_lapse_d,
                            p_strat_trial0 = self.p_strat_trial0.softmax(-1),
                            p_transitions = zero_p_transitions(self.p_transitions),
                            p_strat_responses = p_strat_responses)
        if detach:
            params = params.detach()
        return params
        
    def forward(self):
        """
        In the forward calculation, only the first trial is used (to establish prior)
        """
        num_subjects, num_trials = self.subject_responses.shape
        params = self.get_params(detach=False)

        p_strategies = get_p_strategies(self.subject_responses,
                                        params.p_strat_trial0,
                                        params.p_transitions)
        p_responses = get_p_responses(params.p_strat_responses, p_strategies)
        p_responses_exp = tu.expand_along_dim(p_responses, 0, num_subjects)
        nll = tu.nll_loss(p_responses_exp.log(), self.subject_responses)
        
        return TensorDict(p_strategies=p_strategies,
                          p_responses=p_responses,
                          nll=nll)

In [ ]:
def train_macro_model(subject_responses,
                      num_epochs=1000,
                      lr=.01,
                      show_pbar=False,
                      print_epochs=0):
    
    model = MacroModel(subject_responses)
    model = model.to(subject_responses.device)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    iterable = range(num_epochs + 1)
    
    if show_pbar > 0:
        iterable = tqdm(iterable)
        
    for epoch in iterable:
        optimizer.zero_grad()
        results = model()
        loss = results.nll
        loss.backward()
        optimizer.step()

        if print_epochs > 0 and (epoch%print_epochs == 0 or epoch == num_epochs):
            print("Epoch: {}, NLL: {}".format(epoch, loss))
            print(results.p_strategies[0])
            model.get_params().print(round_digits=4)
            
    return model

# Micromodels

In [ ]:
def fit_micro_models(responses,
                     p_strat_priors,
                     p_strat_responses,
                     p_transitions,
                     show_pbar=False):
    """
    Returns a TensorDict with
        paths: tensor of shape [num_paths, num_trials] indicating the strategy at each step
        p_paths: tensor of shape [num_subjects, num_paths] with probability of each path
    """
    device = responses.device

    # tensor of P(actual response | strategy) for each strategy
    # shape [num_subjects, num_trials, num_strategies]  i.e. [88, 25, 4]
    likelihoods = tu.prepend_shape(p_strat_responses, responses.shape)
    likelihoods = tu.swap_dims(likelihoods, -1, -2)
    likelihoods = tu.select_subtensors(likelihoods, responses)

    # These computations are faster on numpy than pytorch by 3-5x
    p_strat_responses = p_strat_responses.cpu()
    responses = responses.cpu().numpy()
    p_strat_priors = p_strat_priors.cpu().numpy()
    likelihoods = likelihoods.cpu().numpy()
    p_transitions = p_transitions.cpu().numpy()
    
    all_p_paths = []
    # for each subject, build P(path) using Bayes Theorem
    num_samples, num_trials = responses.shape
    iterable = range(num_samples)
    if show_pbar:
        iterable = tqdm(iterable)
    for sid in iterable:
        p_s = likelihoods[sid, 0] * p_strat_priors
        p_s = p_s / np.expand_dims(p_s.sum(-1), axis=-1)
        p_paths = p_s
        paths = [(0,), (1,), (2,), (3,)]

        for trial in range(1, num_trials):
            new_paths = []
            new_p_paths = []

            for i, strat in itertools.product(range(len(paths)), range(4)):
                path = paths[i]

                # reject paths that switch to a worse strategy
                # they need to be left out of the loop so that they aren't included in the normalizing sum (denominator)
                if path[-1] > strat:
                    continue

                # P(path_{t-1}) x P(s | s_{t-1}) x P(r | s)
                p_path = p_paths[i] * p_transitions[path[-1], strat] * likelihoods[sid, trial, strat]
                new_p_paths.append(p_path)

                new_paths.append(path + (strat,))

            paths = new_paths
            p_paths = np.stack(new_p_paths, axis=0)
            p_paths = p_paths / p_paths.sum()

        all_p_paths.append(p_paths)

    all_p_paths = np.stack(all_p_paths, axis=0)
    all_p_paths = torch.tensor(all_p_paths)
    paths = torch.tensor(paths)

    oh_paths = tu.one_hot_encode(paths)
    p_strategies = tu.extend_mul(all_p_paths, oh_paths).sum(1)
    p_responses = tu.extend_mul(p_strategies, p_strat_responses).sum(2)
    return TensorDict(paths=paths,
                      probs=all_p_paths,
                      p_strategies=p_strategies,
                      p_responses=p_responses)

# Load Data

In [ ]:
# Parameters
solvers = True

# Path of data files
puzzle_path = "data/processed/puzzle_data.tsv"
subject_path = "data/processed/subject_data.tsv"

In [ ]:
# Load data

subject_responses, subject_ids = load_responses(puzzle_path,
                                                subject_path,
                                                solvers=solvers,
                                                device=device)

# Create save directory
group = 'solvers' if solvers else 'nonsolvers'
dirpath = proj_path + 'data/hmm/{}/'.format(group)
utils.mkdir(dirpath)

# Train Models

In [ ]:
load = False

if load:
    macro_model = MacroModel(subject_responses)
    macro_model.load_state_dict(torch.load(pickle_path + 'macro_models_{}.mdl'.format(group)))
    macro_model = macro_model.to(device)
    macro_model.eval()
    params = macro_model.get_params()
    micro_models = TensorDict.load(pickle_path + 'micro_models_{}.td'.format(group)).to(device)
else:
    macro_model = train_macro_model(subject_responses, num_epochs=2000, show_pbar=True)
    params = macro_model.get_params()
    micro_models = fit_micro_models(subject_responses,
                                    params.p_strat_trial0,
                                    params.p_strat_responses,
                                    params.p_transitions,
                                    show_pbar=True)

    # Save models
    torch.save(macro_model.state_dict(), pickle_path + 'macro_models_{}.mdl'.format(group))
    micro_models.save(pickle_path + 'micro_models_{}.td'.format(group))

# Macro model posterior distributions

In [ ]:
outputs = macro_model()
df_strategy = TensorDict(probability=outputs.p_strategies).to_dataframe({0: 'trial', 1: 'strategy'})
df_response = TensorDict(probability=outputs.p_responses).to_dataframe({0: 'trial', 1: 'response_type'})
df_strategy.to_csv(dirpath + 'macro_p_strategy.tsv', sep='\t', index=False)
df_response.to_csv(dirpath + 'macro_p_response.tsv', sep='\t', index=False)

# Micro model posterior distributions

In [ ]:
# P(response) and P(strategy) for each participant

p_responses = utils.to_dataframe(micro_model.p_responses, ['subject_id', 'trial', 'response_type'], 'probability')
p_responses.subject_id = [subject_ids[s] for s in p_responses.subject_id]
p_responses.to_csv(dirpath + "subject_p_responses.tsv".format(group), sep='\t', index=False)

p_strategies = utils.to_dataframe(micro_model.p_strategies, ['subject_id', 'trial', 'strategy'], 'probability')
p_strategies.subject_id = [subject_ids[s] for s in p_strategies.subject_id]
p_strategies.to_csv(dirpath + "subject_p_strategies.tsv".format(group), sep='\t', index=False)

In [ ]:
# Top posterior strategy paths for each participant

p_paths, path_idx = micro_model.probs.sort(-1, descending=True)
p_paths = p_paths[:,:5]
path_idx = path_idx[:,:5]

p_paths = utils.to_dataframe(p_paths, ['subject_id', 'rank'], 'probability')
p_paths.subject_id = [subject_ids[s] for s in p_paths.subject_id]
p_paths.to_csv(dirpath + "top_paths_probs.tsv", sep='\t', index=False)

top_paths = []
for i in range(len(subject_responses)):
    idx = path_idx[i]
    top_paths.append(micro_model.paths[idx])
top_paths = torch.stack(top_paths, dim=0)
top_paths = utils.to_dataframe(top_paths, ['subject_id', 'rank', 'trial'], 'strategy')
top_paths.subject_id = [subject_ids[s] for s in top_paths.subject_id]
top_paths.to_csv(dirpath + "top_paths.tsv", sep='\t', index=False)

# Gradual vs Discrete Transitions

In [ ]:
def sample_strat0(p_strat_trial0, num_samples, include_hs=False):
    if not include_hs:
        p_strat_trial0 = tu.normalize(p_strat_trial0[:3])
    strats = Categorical(p_strat_trial0).sample((num_samples, ))
    return tu.one_hot_encode(strats, 4).float()

def get_expected_num_strategy_observations(mean_p_strat_trial0,
                                           mean_p_transitions,
                                           num_subjects,
                                           num_trials):
    obs = [mean_p_strat_trial0.unsqueeze(0)]
    for i in range(num_trials):
        obs.append(obs[-1].mm(mean_p_transitions))
    obs = torch.cat(obs, 0)
    obs = (num_subjects * obs).sum(0)
    return obs

def get_p_strat_responses_dirichlet_params(mean_p_strat_trial0,
                                           mean_p_transitions,
                                           mean_p_strat_responses,
                                           num_subjects,
                                           num_trials):
    obs = get_expected_num_strategy_observations(mean_p_strat_trial0,
                                                 mean_p_transitions,
                                                 num_subjects,
                                                 num_trials)
    return obs.unsqueeze(-1) * mean_p_strat_responses

def get_p_transitions_dirichlet_params(mean_p_strat_trial0, mean_p_transitions, num_subjects, num_trials):
    obs = get_expected_num_strategy_observations(mean_p_strat_trial0,
                                                 mean_p_transitions,
                                                 num_subjects,
                                                 num_trials)
    return obs.unsqueeze(-1) * mean_p_transitions 

def sample_p_strat_responses(mean_p_strat_trial0,
                             mean_p_transitions,
                             mean_p_strat_responses,
                             num_subjects,
                             num_trials,
                             num_samples):
    params = get_p_strat_responses_dirichlet_params(mean_p_strat_trial0,
                                                    mean_p_transitions,
                                                    mean_p_strat_responses,
                                                    num_subjects,
                                                    num_trials)
    samples = Dirichlet(params).sample((num_samples, ))
    samples = tu.round(samples, 8, keep_device=True) # to 0 out near-0 probabilities
    return tu.normalize(samples)

def sample_p_transitions(mean_p_strat_trial0, mean_p_transitions, num_subjects, num_trials, num_samples):
    params = get_p_transitions_dirichlet_params(mean_p_strat_trial0, mean_p_transitions, num_subjects, num_trials)
    samples = Dirichlet(params).sample((num_samples, ))
    samples = tu.round(samples, 8, keep_device=True) # to 0 out near-0 probabilities
    return tu.normalize(samples)

def sample_p_strategies(p_strat0, p_transitions, discrete, num_trials):
    """
    p_strat0: tensor of shape [num_samples, 4]
    p_transitions: tensor of shape [num_samples, 4, 4]
    discrete: if True, argmax's the strategy at each trial
    """
    strategies = [p_strat0]
    for i in range(num_trials - 1):
        strat = strategies[-1].unsqueeze(1).bmm(p_transitions).squeeze(1)
        if discrete:
            strat = tu.one_hot_encode(Categorical(strat).sample(), 4).float()
        strategies.append(strat)
    strategies = torch.stack(strategies, 1)
    return strategies

def get_p_responses(p_strategies, p_strat_responses):
    p_strategies = p_strategies.unsqueeze(-2)
    p_strat_responses = tu.expand_along_dim(p_strat_responses, 1, p_strategies.shape[1]).contiguous()
    p_responses = tu.bmm(p_strategies, p_strat_responses).squeeze(-2)
    return p_responses

def sample_responses(p_strategies, p_strat_responses):
    p_responses = get_p_responses(p_strategies, p_strat_responses)
    return Categorical(p_responses).sample()

def get_p_response_sequence(p_strategies, p_strat_responses, subject_responses):
    num_subjects = subject_responses.shape[0]
    num_samples = p_strategies.shape[0]
    p_responses = get_p_responses(p_strategies, p_strat_responses)
    p_responses = tu.expand_along_dim(p_responses, 1, num_subjects)
    subject_responses = tu.prepend_shape(subject_responses, num_samples)
    log_prob = Categorical(p_responses).log_prob(subject_responses)
    prob = log_prob.sum(-1).exp().mean(0)
    return prob

def get_bayes_factor(p_response_seq1, p_response_seq2):
    p1 = .5

    for i in np.random.permutation(len(p_response_seq1)):
        num = p_response_seq1[i] * p1
        den = num + (p_response_seq2[i] * (1 - p1))
        p1 = num / den
    
    return p1 / (1 - p1)

## With Dirichlet distributions

In [ ]:
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [ ]:
# Simulate responses
num_samples = 100
num_trials = 25
num_subjects = 88
p_strat0 = sample_strat0(params.p_strat_trial0, num_samples, include_hs=True)
p_transitions = sample_p_transitions(mean_p_strat_trial0=params.p_strat_trial0,
                                     mean_p_transitions=params.p_transitions,
                                     num_subjects=num_subjects,
                                     num_trials=num_trials,
                                     num_samples=num_samples)
p_strat_responses = sample_p_strat_responses(mean_p_strat_trial0=params.p_strat_trial0,
                                             mean_p_transitions=params.p_transitions,
                                             mean_p_strat_responses=params.p_strat_responses,
                                             num_subjects=num_subjects,
                                             num_trials=num_trials,
                                             num_samples=num_samples)
p_disc_strategies = sample_p_strategies(p_strat0, p_transitions, True, num_trials)
p_grad_strategies = sample_p_strategies(p_strat0, p_transitions, False, num_trials)

# 100000 doesn't fit on GPU
p_disc_strategies = p_disc_strategies.to('cpu')
p_grad_strategies = p_grad_strategies.to('cpu')
p_strat_responses = p_strat_responses.to('cpu')
s_responses = subject_responses.to('cpu')

In [ ]:
disc_p_response_seq = get_p_response_sequence(p_disc_strategies, p_strat_responses, s_responses)
grad_p_response_seq = get_p_response_sequence(p_grad_strategies, p_strat_responses, s_responses)
bayes_factor = get_bayes_factor(disc_p_response_seq, grad_p_response_seq)
utils.kv_print(Bayes_factor=bayes_factor)

## Without Dirichlet Distributions

In [ ]:
# Simulate responses
num_samples = 100
num_trials = 25
num_subjects = 88
params = macro_model.get_params()
p_strat0 = sample_strat0(params.p_strat_trial0, num_samples, include_hs=True)
p_transitions = tu.prepend_shape(params.p_transitions, num_samples)
p_strat_responses = tu.prepend_shape(params.p_strat_responses, num_samples)
p_disc_strategies = sample_p_strategies(p_strat0, p_transitions, True, num_trials)
p_grad_strategies = sample_p_strategies(p_strat0, p_transitions, False, num_trials)

# 100000 doesn't fit on GPU
p_disc_strategies = p_disc_strategies.to('cpu')
p_grad_strategies = p_grad_strategies.to('cpu')
p_strat_responses = p_strat_responses.to('cpu')
s_responses = subject_responses.to('cpu')

In [ ]:
disc_p_response_seq = get_p_response_sequence(p_disc_strategies, p_strat_responses, s_responses)
grad_p_response_seq = get_p_response_sequence(p_grad_strategies, p_strat_responses, s_responses)
bayes_factor = get_bayes_factor(disc_p_response_seq, grad_p_response_seq)
utils.kv_print(Bayes_factor=bayes_factor)

In [ ]:
# df_strategies = TensorDict(p_discrete=p_disc_strategies
td_strategies = TensorDict(p=torch.stack([p_disc_strategies, p_grad_strategies], 0))
df_strategies = td_strategies.to_dataframe({0: 'xtype', 1: 'sim_id', 2: 'trial', 3: 'strategy'})
df_strategies.xtype = ['discrete' if t == 0 else 'gradual' for t in df_strategies.xtype]
df_strategies.to_csv(dirpath + "xtype_sim_strategies.tsv", sep='\t', index=False)

# Model Accuracy

In [ ]:
# Simulate responses
num_samples = 1000
num_trials = 25
num_subjects = 88

params = macro_model.get_params()
p_strat0 = sample_strat0(params.p_strat_trial0, num_samples, include_hs=True)
p_transitions = sample_p_transitions(mean_p_strat_trial0=params.p_strat_trial0,
                                     mean_p_transitions=params.p_transitions,
                                     num_subjects=num_subjects,
                                     num_trials=num_trials,
                                     num_samples=num_samples)
p_strat_responses = sample_p_strat_responses(mean_p_strat_trial0=params.p_strat_trial0,
                                             mean_p_transitions=params.p_transitions,
                                             mean_p_strat_responses=params.p_strat_responses,
                                             num_subjects=num_subjects,
                                             num_trials=num_trials,
                                             num_samples=num_samples)
p_disc_strategies = sample_p_strategies(p_strat0, p_transitions, True, num_trials)
disc_responses = sample_responses(p_disc_strategies, p_strat_responses)

In [ ]:
disc_micro_model = fit_micro_models(disc_responses,
                                    params.p_strat_trial0,
                                    params.p_strat_responses,
                                    params.p_transitions,
                                    show_pbar=True)
disc_micro_model.responses = disc_responses.cpu()
disc_micro_model.strategies_oh = p_disc_strategies.cpu()
disc_micro_model.strategies = disc_micro_model.strategies_oh.argmax(-1)
disc_micro_model.save(pickle_path + 'disc_micro_model.td')

In [ ]:
# Save strategy inferences
td = TensorDict(p_actual=disc_micro_model.strategies_oh,
                p_predicted=disc_micro_model.p_strategies)
df = td.to_dataframe({0: 'sim', 1: 'trial', 2: 'strategy'})
df.to_csv(dirpath + "sim_strategy_accuracy.tsv".format(group), sep='\t', index=False)

# Parameters

In [ ]:
params = macro_model.get_params()
params.print(round_digits=3)

In [ ]:
transitions = get_p_transitions_dirichlet_params(params.p_strat_trial0,
                                                 params.p_transitions,
                                                 num_subjects=88,
                                                 num_trials=25)
tu.round(transitions, 3, keep_device=False)

In [ ]:
emissions = get_p_strat_responses_dirichlet_params(params.p_strat_trial0,
                                                   params.p_transitions,
                                                   params.p_strat_responses,
                                                   num_subjects=88,
                                                   num_trials=25)
tu.round(emissions, 3, keep_device=False)